# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

from api_keys import weather_api_key
from citipy import citipy

In [18]:

#Casts a dataframe with two very long sets of random numbers formatted for use as random lat/long coordinates
cities_df = pd.DataFrame({"long":(np.random.uniform(-1.8,1.8,1200)*100), "lat":(np.random.uniform(-0.9,0.9,1200)*100)})

####
#### API SAVER
####
cities_df = cities_df.truncate(after = 9)


cities_df.head()

#city = citipy.nearest_city(22.99,120.21)
#city.country_code

,long,lat
0,-112.802827,-33.989944
1,37.147355,-16.016811
2,107.019522,-8.241888
3,132.027680,-25.183759
4,-71.050513,27.173410


## Generate Cities List

In [19]:
cities_name = []
countrycode = []
for i in range(0,len(cities_df['lat'])):
    
    #API saver
    if i>10:
        break
    
    
    lat = cities_df.iloc[i]['lat']
    long = cities_df.iloc[i]['long']
    city = citipy.nearest_city(lat,long)
    
    name = city.city_name
    cities_name.append(name)
    country = city.country_code
    countrycode.append(country)
    
cities_df['City'] = cities_name
cities_df['Country'] = countrycode
cities_df.head(10)

,long,lat,City,Country
0,-112.802827,-33.989944,rikitea,pf
1,37.147355,-16.016811,cuamba,mz
2,107.019522,-8.241888,banjar,id
3,132.027680,-25.183759,yulara,au
4,-71.050513,27.173410,cockburn town,bs
5,-35.724540,-78.524171,ushuaia,ar
6,18.592361,39.851787,tricase,it
7,-30.428266,16.377479,ponta do sol,cv
8,41.398823,18.492719,abha,sa
9,152.949628,11.330771,kavieng,pg


In [20]:
#drop duplicate cities in cities_df
cities_copy = cities_df.copy()
new_cities = cities_copy.drop_duplicates("City", keep = 'first')

#reset cities_df's index and truncate at 499 to provide sample size of 500 random cities.
new_cities = new_cities.reset_index()
new_cities = new_cities.truncate( after=499)
new_cities

,index,long,lat,City,Country
0,0,-112.802827,-33.989944,rikitea,pf
1,1,37.147355,-16.016811,cuamba,mz
2,2,107.019522,-8.241888,banjar,id
3,3,132.027680,-25.183759,yulara,au
4,4,-71.050513,27.173410,cockburn town,bs
5,5,-35.724540,-78.524171,ushuaia,ar
6,6,18.592361,39.851787,tricase,it
7,7,-30.428266,16.377479,ponta do sol,cv
8,8,41.398823,18.492719,abha,sa
9,9,152.949628,11.330771,kavieng,pg


In [21]:
api_key = weather_api_key


#Establish columns for data to be used in scatter plots
new_cities["Temp"]=""
new_cities["Humidity"]=""
new_cities["Cloudiness"]=""
new_cities["Wind (mph)"]=""
new_cities.head(10)

,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-112.802827,-33.989944,rikitea,pf,,,,
1,1,37.147355,-16.016811,cuamba,mz,,,,
2,2,107.019522,-8.241888,banjar,id,,,,
3,3,132.027680,-25.183759,yulara,au,,,,
4,4,-71.050513,27.173410,cockburn town,bs,,,,
5,5,-35.724540,-78.524171,ushuaia,ar,,,,
6,6,18.592361,39.851787,tricase,it,,,,
7,7,-30.428266,16.377479,ponta do sol,cv,,,,
8,8,41.398823,18.492719,abha,sa,,,,
9,9,152.949628,11.330771,kavieng,pg,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [22]:
#Use iterrows() to loop through the new_cities sample size and perform API calls for each 

In [23]:
#Test API block status

target_url=f"http://api.openweathermap.org/data/2.5/weather?q={new_cities.City[4]}&appid={weather_api_key}"
weather_json = requests.get(target_url).json()
print(weather_json)
#print(weather_api_key)
#print("       ")
#print("       ")
#print(weather_json["main"]["temp"])


#temp_list.append(weather_json["main"]["temp"])
#print(temp_list)

{'coord': {'lon': -71.14, 'lat': 21.46}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 302.15, 'feels_like': 303.56, 'temp_min': 302.15, 'temp_max': 302.15, 'pressure': 1014, 'humidity': 74}, 'visibility': 10000, 'wind': {'speed': 6.2, 'deg': 80}, 'clouds': {'all': 40}, 'dt': 1599770872, 'sys': {'type': 1, 'id': 7041, 'country': 'TC', 'sunrise': 1599733833, 'sunset': 1599778350}, 'timezone': -14400, 'id': 3576994, 'name': 'Cockburn Town', 'cod': 200}


In [24]:
temp_list = []
humid_list = []
cloud_list = []
wind_list = []

for index, row in new_cities.iterrows():
    
    
    #API saver
    if index>10:
        break
    
    target_url=f"http://api.openweathermap.org/data/2.5/weather?q={row[3]}&appid={weather_api_key}"
    weather_json = requests.get(target_url).json()
    print(f"Processing Record {index} | {row[3]}")
    
    try:
        #new_cities.set_value(index, "Temp", weather_json["main"]["temp"])
        temp_list.append(weather_json["main"]["temp"])
        humid_list.append(weather_json["main"]["humidity"])
        cloud_list.append(weather_json["clouds"]["all"])
        wind_list.append(weather_json["wind"]["speed"])
        
        
        #add in list appends here for Humidity, Cloudiness, and Wind (mph)
        
        
        
        
        #new_cities.set_value(index, "Humidity", weather_json["main"]["humidity"])
        #new_cities.set_value(index, "Cloudiness", weather_json["clouds"]["all"])
        #new_cities.set_value(index, "Wind (mph)", weather_json["wind"]["speed"])
        
    except:
        temp_list.append("")
        humid_list = ("")
        cloud_list = ("")
        wind_list = ("")
        print("City not found. Skipping...")

Processing Record 0 | rikitea
Processing Record 1 | cuamba
Processing Record 2 | banjar
Processing Record 3 | yulara
Processing Record 4 | cockburn town
Processing Record 5 | ushuaia
Processing Record 6 | tricase
Processing Record 7 | ponta do sol
Processing Record 8 | abha
Processing Record 9 | kavieng


In [25]:
new_cities["Temp"] = temp_list
new_cities["Humidity"] = humid_list
new_cities["Cloudiness"] = cloud_list
new_cities["Wind (mph)"] = wind_list
###
### ADD A DROPNA STATEMENT DOWN BELOW TO REMOVE ALL MISSING OPTIONS
### OR DELETE BASED ON TEMP BEING "NAN" OR "" OR WHATEVER
### JUST MAKE SURE YOU DO IT
###

#print(len(temp_list))
 

def kelv_to_fahr(temp_kelv):
    """Convert Kelvin to Fahrenheit
    
    Return Fahrenheit conversion of input"""
    
    temp_fahr = (temp_kelv)* 9/5 - 459.67
    return temp_fahr


#converts the strings to numbers so they can be formatted into fahrenheit
#new_cities.dropna() to get rid of the empty variables where data was lacking
#or do a reverse loc statement, removing all that has this characteristic - "NaN" or "" 
#before numericization

#To remove all rows where column 'score' is < 50:
#df = df.drop(df[df.score < 50].index)
#new_cities = new_cities.drop(new_cities[new_cities.Temp == ""].index)

new_cities["Temp"] = pd.to_numeric(new_cities["Temp"])
new_cities["Temp"] = kelv_to_fahr(new_cities["Temp"])



new_cities.head()

C:\Users\riggi\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-112.802827,-33.989944,rikitea,pf,73.328,84,100,7.68
1,1,37.147355,-16.016811,cuamba,mz,68.774,57,45,1.44
2,2,107.019522,-8.241888,banjar,id,78.800,100,20,4.60
3,3,132.027680,-25.183759,yulara,au,60.800,35,0,2.10
4,4,-71.050513,27.173410,cockburn town,bs,84.200,74,40,6.20


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [26]:
#export dataframe as csv
new_cities.head()

,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-112.802827,-33.989944,rikitea,pf,73.328,84,100,7.68
1,1,37.147355,-16.016811,cuamba,mz,68.774,57,45,1.44
2,2,107.019522,-8.241888,banjar,id,78.800,100,20,4.60
3,3,132.027680,-25.183759,yulara,au,60.800,35,0,2.10
4,4,-71.050513,27.173410,cockburn town,bs,84.200,74,40,6.20


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [27]:
#To remove all rows where column 'score' is < 50:
#df = df.drop(df[df.score < 50].index)

new_cities = new_cities.drop(new_cities[new_cities.Humidity > 100].index)
new_cities.head()

,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-112.802827,-33.989944,rikitea,pf,73.328,84,100,7.68
1,1,37.147355,-16.016811,cuamba,mz,68.774,57,45,1.44
2,2,107.019522,-8.241888,banjar,id,78.800,100,20,4.60
3,3,132.027680,-25.183759,yulara,au,60.800,35,0,2.10
4,4,-71.050513,27.173410,cockburn town,bs,84.200,74,40,6.20


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression